In [1]:
import iris
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import iris.quickplot as qplt
import iris.plot as iplt
import itertools
import datetime
import glob
import os, sys
import cartopy.crs as ccrs
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
forecast_date_time = datetime.datetime(2020, 11, 1, 12)
str_year, str_month, str_day, str_hour = str(forecast_date_time.year), \
                                             str('%02d' % forecast_date_time.month), \
                                             str('%02d' % forecast_date_time.day), \
                                             str('%02d' % forecast_date_time.hour)

date_label = '%s%s%s_%sZ' % (str_year, str_month, str_day, str_hour)

In [3]:
#out_data_dir = os.path.join(data_paths.dirs('mog_forecast_out_dir'), str_year, str_month, str_day)
out_data_dir = os.path.join('/scratch/hadpx/cold_surge_monitoring/mogreps/processed_data', str_year, str_month, str_day, str_hour)

In [4]:
# Read the processed and combined data
precip_file_name = os.path.join(out_data_dir, 'MOG_PRECIP_24H_%s.nc' % (date_label))
precip_cube = iris.load_cube(precip_file_name)

u850_file_name = os.path.join(out_data_dir, 'MOG_x_wind_850_24H_%s.nc' % (date_label))
u850_cube = iris.load_cube(u850_file_name)

v850_file_name = os.path.join(out_data_dir, 'MOG_y_wind_850_24H_%s.nc' % (date_label))
v850_cube = iris.load_cube(v850_file_name)

In [5]:
speed_cube = (u850_cube**2 + v850_cube**2)**0.5

In [6]:
precip_ens_mean = precip_cube.collapsed('realization', iris.analysis.MEAN)
u850_ens_mean = u850_cube.collapsed('realization', iris.analysis.MEAN)
v850_ens_mean = v850_cube.collapsed('realization', iris.analysis.MEAN)
speed_ens_mean = speed_cube.collapsed('realization', iris.analysis.MEAN)

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'realization'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'forecast_reference_time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'realization'.
  warnings.warn(msg.format(self.name()))
/opt/

In [7]:
qplt.contourf(speed_ens_mean[0])

In [13]:
xlon = u850_cube.coord('longitude').points
ylat = u850_cube.coord('latitude').points
X, Y = np.meshgrid(xlon, ylat)

In [14]:
X.shape, u850_cube.shape, v850_cube.shape

((214, 178), (8, 36, 214, 178), (8, 36, 214, 178))

In [9]:
t = 2 # for t in times:

valid_date = forecast_date_time + datetime.timedelta(days=t)
valid_date_label = '%s/%s/%s' % (valid_date.year, str('%02d' % valid_date.month), str('%02d' % valid_date.day))
        
lw = 3 * speed_ens_mean[t].data / 20.0 #speed.data.max()

fig = plt.figure(figsize=(10, 10), dpi=100)

# contour plot of precip
clevels = [1, 5, 10, 15, 20, 25, 30, 35, 40]
cf = iplt.contourf(precip_ens_mean[t], levels=clevels, cmap='gray_r', alpha=0.65, extend='max')
axc = plt.gca()
axc.coastlines('50m', alpha=0.5)
axc.set_ylim([-10, 20])
axc.set_xlim([90, 130])
axc.stock_img()
axc.set_yticks([-10, 0, 10, 20], crs=ccrs.PlateCarree())
axc.set_xticks([90, 100, 110, 120, 130], crs=ccrs.PlateCarree())
#axc.gridlines()

plt.title('Ensemble mean PREC, 850hPa winds \n Forecast start: %s, Lead: T+%s h \n Valid at %s' % (date_label, (t * 24), valid_date_label))

bounds = np.arange(2,22,2)
norm = colors.BoundaryNorm(boundaries=bounds, ncolors=256)
    
# overplot wind as streamlines
cl = plt.gca().streamplot(X, Y, u850_ens_mean[t].data, v850_ens_mean[t].data, 
                          density=(2., 1.5), maxlength=1, norm=norm,
                          cmap='GnBu',
                          color=speed_ens_mean[t].data, linewidth=lw)

# these are matplotlib.patch.Patch properties
props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

font = {'color':  'red', 'weight': 'bold', 'size': 12}
axc.text(91, 19, "COLD SURGE: {:.1f}%".format(cs_prob[t]*100.), fontdict=font, verticalalignment='top', bbox=props, alpha=0.7)

font = {'color':  'red', 'weight': 'bold', 'size': 12}
axc.text(91, 17.5, "CROSS-EQUATORIAL SURGE: {:.1f}%".format(ces_prob[t]*100.), fontdict=font, verticalalignment='top', bbox=props, alpha=0.7)
        
colorbar_axes = plt.gcf().add_axes([0.2, 0.12, 0.6, 0.025])
colorbar = plt.colorbar(cf, colorbar_axes, orientation='horizontal', label='Precip (mm day$^{-1}$)')

colorbar_axes = plt.gcf().add_axes([0.92, 0.25, 0.025, 0.5])
colorbar = plt.colorbar(cl.lines, colorbar_axes, orientation='vertical', label='Wind Speed (m s$^{-1}$)')

NameError: name 'X' is not defined

In [12]:
print("COLD SURGE: {:.1f}%".format(cs_prob[t]*100.))

NameError: name 'cs_prob' is not defined

In [188]:
for t in np.arange(ntimes):
    valid_date = date + datetime.timedelta(days=int(t))
    print(valid_date)

2021-01-01 12:00:00
2021-01-02 12:00:00
2021-01-03 12:00:00
2021-01-04 12:00:00
2021-01-05 12:00:00
2021-01-06 12:00:00
2021-01-07 12:00:00
2021-01-08 12:00:00


In [72]:
result = precip_cube.collapsed('realization', iris.analysis.PERCENTILE, percent=[10])
qplt.contourf(result[2])
plt.gca().coastlines()

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'realization'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'forecast_reference_time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))


In [74]:
result = speed_cube.collapsed('realization', iris.analysis.PERCENTILE, percent=[90])
qplt.contourf(result[2])
plt.gca().coastlines()

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'realization'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'forecast_reference_time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))


In [36]:
t = 1
precip_threshold = 50
speed_threshold = 10
precip_prob = precip_cube.collapsed('realization', iris.analysis.PROPORTION, 
                               function=lambda values: values > precip_threshold)

speed_prob = speed_cube.collapsed('realization', iris.analysis.PROPORTION, 
                               function=lambda values: values > speed_threshold)

fig = plt.figure(figsize=(10, 10), dpi=100)

try:
    # contour plot of precip
    clevels = np.arange(0.1, 1.1, 0.1)
    cf = iplt.contourf(precip_prob[t], levels=clevels, cmap='gray_r', alpha=0.65, extend='max')
    axc = plt.gca()
except:
    axc = plt.subplot(111, projection=ccrs.PlateCarree())
    
axc.coastlines('50m', alpha=0.5)
axc.set_ylim([-10, 20])
axc.set_xlim([90, 130])
axc.stock_img()
axc.set_yticks([-10, 0, 10, 20], crs=ccrs.PlateCarree())
axc.set_xticks([90, 100, 110, 120, 130], crs=ccrs.PlateCarree())

'''
axc = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
#axc.set_extent([90, 130, -10, 20], crs=ccrs.PlateCarree())
axc.coastlines('50m', alpha=0.5)
axc.set_ylim([-10, 20])
axc.set_xlim([90, 130])
axc.set_yticks([-10, 0, 10, 20], crs=ccrs.PlateCarree())
axc.set_xticks([90, 100, 110, 120, 130], crs=ccrs.PlateCarree())
axc.stock_img()
'''
#axc.gridlines()

clevels = np.arange(0.5, 1.01, 0.1)
cfc = iplt.contour(speed_prob[t], levels=clevels, linewidths=clevels*3, extend='max', alpha=0.7)
#plt.gca().coastlines()

title = 'Ensemble probability PREC, 850hPa winds \n Forecast start: %s, Lead: T+%s h \n Valid at %s' % (date_label, (t * 24), valid_date_label)
#title += '\n Precip. prob (p$\geq$ %s mm day$^{-1}$)' %precip_threshold
#title += '\n Wind speed. prob (p$\geq$ %s m s$^{-1}$)' %speed_threshold

plt.title(title)
colorbar_axes = plt.gcf().add_axes([0.2, 0.12, 0.6, 0.025])
colorbar = plt.colorbar(cf, colorbar_axes, orientation='horizontal', label='Precip. prob (p$\geq$ %s mm day$^{-1}$)' %precip_threshold)

colorbar_axes = plt.gcf().add_axes([0.91, 0.2, 0.025, 0.2])
cb2 = plt.colorbar(cfc, colorbar_axes, orientation='vertical', label='Wind speed. prob (p$\geq$ %s m s$^{-1}$)' %speed_threshold)
cb2.outline.set_edgecolor('white')
# set colorbar ticklabels
cb2.ax.tick_params(color="white")

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'realization'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'forecast_reference_time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'realization'.
  warnings.warn(msg.format(self.name()))
/opt/

In [32]:
precip_prob = precip_cube.collapsed('realization', iris.analysis.PROPORTION, 
                               function=lambda values: values > 2000)

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'realization'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'forecast_reference_time'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1406: UserWarning: Collapsing a multi-dimensional coordinate. Metadata may not be fully descriptive for 'time'.
  warnings.warn(msg.format(self.name()))


In [36]:
iplt.contourf(precip_prob[3])

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/cartopy/mpl/geoaxes.py:1366: UserWarning: No contour levels were found within the data range.
  result = matplotlib.axes.Axes.contour(self, *args, **kwargs)


In [39]:
#mask = u850_ens_mean[0].data.copy()
dummy = speed_ens_mean.copy()
mask1 = (u850_ens_mean.data >= 0.) #& 
mask2 = (v850_ens_mean.data >= 0.) #& 
mask3 = (speed_ens_mean.data >= 5.)
#mask = (speed_ens_mean.data < 10.)
dummy.data = np.ma.array(speed_ens_mean.data, mask=mask1*mask2*mask3)
qplt.contourf(dummy[0])
plt.gca().coastlines()

In [85]:
for member in range(10):
    t = 1
    plt.quiver(X[:1,:1], Y[:1,:1], u850_cube[t,member, :1,:1].data, v850_cube[t,member, :1,:1].data, alpha=0.2)

In [83]:
#u850_cube

X Wind (m s-1),forecast_period,realization,latitude,longitude
Shape,8,36,214,178
Dimension coordinates,,,,
forecast_period,x,-,-,-
realization,-,x,-,-
latitude,-,-,x,-
longitude,-,-,-,x
Auxiliary coordinates,,,,
forecast_reference_time,x,x,-,-
time,x,x,-,-
Scalar coordinates,,,,


In [455]:
def enumerated_product(*args):
    yield from zip(itertools.product(*(range(len(x)) for x in args)), itertools.product(*args))
    

def cold_surge_probabilities(u850_cube, v850_cube, speed_cube):
    # Cold surge identification
    chang_box = [107, 115, 5, 10]        # CP Chang's 2nd domain

    # Hattori box for cross equatorial surges
    hattori_box = [105, 115, -5, 5]

    Chang_threshold = 9.0 # 10 # wind speed m/s
    Hattori_threshold = -2.0 # m/s meridional wind
    
    u850_ba = u850_cube.intersection(latitude=(chang_box[2], chang_box[3]), longitude=(chang_box[0], chang_box[1]))
    u850_ba = u850_ba.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)

    v850_ba = v850_cube.intersection(latitude=(chang_box[2], chang_box[3]), longitude=(chang_box[0], chang_box[1]))
    v850_ba = v850_ba.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)

    speed_ba = speed_cube.intersection(latitude=(chang_box[2], chang_box[3]), longitude=(chang_box[0], chang_box[1]))
    speed_ba = speed_ba.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)
    
    # Hattori index
    v850_hattori = v850_cube.intersection(latitude=(hattori_box[2], hattori_box[3]), longitude=(hattori_box[0], hattori_box[1]))
    v850_hattori = v850_hattori.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)

    # extract the forecast periods and members from the data
    # to create a metrics array
    forecast_periods = u850_cube.coord('forecast_period').points
    members = u850_cube.coord('realization').points
    
    # Check for cross-equatorial surges
    # CP index
    mask1 = u850_ba.data>0.
    mask2 = v850_ba.data>0.
    mask3 = speed_ba.data<=Chang_threshold
    mask4 = v850_ba.data>Hattori_threshold

    speed_ma = speed_ba.data.copy()
    speed_ma = np.ma.array(speed_ma, mask=mask1)
    speed_ma = np.ma.array(speed_ma, mask=mask2)
    cs_metric = np.ma.array(speed_ma, mask=mask3)

    #CES Hattori index
    ces_metric = np.ma.array(v850_ba.data, mask=mask4)
    ces_metric = np.ma.array(ces_metric, mask=cs_metric.mask)
    
    # return the probabilities as fraction
    return cs_metric.count(axis=1)/float(len(members)), ces_metric.count(axis=1)/float(len(members))

In [458]:
cs_prob, ces_prob = cold_surge_probabilities(u850_cube, v850_cube, speed_cube)

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/cube.py:3218: UserWarning: Collapsing spatial coordinate 'latitude' without weighting
  warnings.warn(msg.format(coord.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'latitude'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'longitude'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/cube.py:3218: UserWarning: Collapsing spatial coordinate 'latitude' without weighting
  warnings.warn(msg.format(coord.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: User

In [468]:
plt.plot(cs_prob*100.)
plt.plot(ces_prob*100.)

In [382]:
u850_ba = u850_cube.intersection(latitude=(chang_box[2], chang_box[3]), longitude=(chang_box[0], chang_box[1]))
u850_ba = u850_ba.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)

v850_ba = v850_cube.intersection(latitude=(chang_box[2], chang_box[3]), longitude=(chang_box[0], chang_box[1]))
v850_ba = v850_ba.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)

speed_ba = speed_cube.intersection(latitude=(chang_box[2], chang_box[3]), longitude=(chang_box[0], chang_box[1]))
speed_ba = speed_ba.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)

# Hattori index
v850_hattori = v850_cube.intersection(latitude=(hattori_box[2], hattori_box[3]), longitude=(hattori_box[0], hattori_box[1]))
v850_hattori = v850_hattori.collapsed(('latitude', 'longitude'), iris.analysis.MEAN)

/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/cube.py:3218: UserWarning: Collapsing spatial coordinate 'latitude' without weighting
  warnings.warn(msg.format(coord.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'latitude'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: UserWarning: Collapsing a non-contiguous coordinate. Metadata may not be fully descriptive for 'longitude'.
  warnings.warn(msg.format(self.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/cube.py:3218: UserWarning: Collapsing spatial coordinate 'latitude' without weighting
  warnings.warn(msg.format(coord.name()))
/opt/scitools/environments/default/2020_10_12/lib/python3.6/site-packages/iris/coords.py:1410: User

In [88]:
Chang_threshold = 9.0 # 10 # wind speed m/s
Hattori_threshold = -2.0 # m/s meridional wind

mask1 = u850_ens_mean.data>0.
mask2 = v850_ens_mean.data>0.
mask3 = speed_ens_mean.data<=Chang_threshold
mask4 = v850_ens_mean.data>Hattori_threshold

#np.ma.array(speed_ba.data, mask=)
speed_ma = speed_ens_mean.data.copy()

speed_ma = np.ma.array(speed_ma, mask=mask1)
speed_ma = np.ma.array(speed_ma, mask=mask2)

cs_metric = speed_ens_mean.copy()
cs_metric.data = np.ma.array(speed_ma, mask=mask3)

plt.subplot(111)
qplt.pcolormesh(cs_metric[0])
plt.gca().coastlines()

In [92]:
#CES
speed_ces = speed_ens_mean.data.copy()
speed_ces = np.ma.array(speed_ces, mask=mask1)
speed_ces = np.ma.array(speed_ces, mask=mask2)
#speed_ces = np.ma.array(speed_ces, mask=mask3)
speed_ces = np.ma.array(speed_ces, mask=mask4)

ces_metric = speed_ens_mean.copy()
ces_metric.data = np.ma.array(speed_ces, mask=mask4)
#ces_metric.data = np.ma.array(ces_metric.data, mask=cs_metric.data.mask)

plt.subplot(111)
qplt.pcolormesh(ces_metric[2])
plt.gca().coastlines()
#plt.colorbar()

In [51]:
speed_ma.shape

(8, 214, 178)

In [454]:
plt.plot(cs_metric.count(axis=1)/36.)
plt.plot(ces_metric.count(axis=1)/36.)

In [21]:
def plot_data(ucube, vcube, precip, plot):

    lons = ucube.coord('longitude').points
    lats = ucube.coord('latitude').points

    # flip data around dateline
    # lons = np.array(lons)
    # ind = np.where(lons <= 180.)[0][-1]
    # print ind, lons[ind]
    # cube1 = olr_cube.copy()
    # cube1.data[:, :] = 0.
    # cube1.data[:, :ind] = olr_cube.data[:, ind:]
    # cube1.data[:, ind:] = olr_cube.data[:, :ind]

    # lons[np.where(lons > 180.)] = lons[np.where(lons > 180.)] - 360.
    # cube1.coord('longitude').points = np.sort(lons)
    # Linear
    color_mapper = LinearColorMapper(palette=GnBu9, low=0, high=50)
    # Log mapper
    #color_mapper = LogColorMapper(palette=GnBu9, low=10, high=300)

    # coastlines
    #with open(os.path.join(os.path.dirname(__file__), 'data/countries.geo.json'), 'r') as f:
    with open('../bokeh_display/data/countries.geo.json', 'r') as f:
        countries = GeoJSONDataSource(geojson=f.read())

    plot.image(image=[precip.data], x=70, y=-20, dw=80, dh=50, color_mapper=color_mapper, alpha=0.5)
    plot.patches("xs", "ys", color=None, line_color="black", source=countries)

    # Vectors
    x0, y0, x1, y1, xR, yR, xL, yL, length = winds.arrows(lons, lats, ucube.data, vcube.data,
                                                          density=5, maxspeed=5, arrowLength=2, arrowHeadAngle=10)

    #cm = np.array(["#C7E9B4", "#7FCDBB", "#41B6C4", "#1D91C0", "#225EA8", "#0C2C84"])
    cm = np.array(Magma6)
    ix = ((length - length.min()) / (length.max() - length.min()) * 5).astype('int')

    colors = cm[ix]

    #colors = 'black'
    plot.segment(x0, y0, x1, y1, color=colors, line_width=1, alpha=0.5)
    plot.segment(x1, y1, xR, yR, color=colors, line_width=1, alpha=0.5)
    plot.segment(x1, y1, xL, yL, color=colors, line_width=1, alpha=0.5)

    plot.background_fill_color = "white"
    plot.x_range = Range1d(start=80, end=150)
    plot.y_range = Range1d(start=-20, end=30)

    title = 'Winds 850, precip Forecast reference time: %s Forecast period: %s H Valid on: %s'\
            %(str(ucube.coord('forecast_reference_time'))[10:29],
              str(ucube.coord('forecast_period').points[0]),
              str(ucube.coord('time'))[10:20])
    plot.title.text = title
    plot.title.text_font_size = "17px"

    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=12,
                         border_line_color=None, location=(0, 0),
                         orientation='horizontal')
    return plot, color_bar

In [29]:
from bokeh.layouts import row, column, widgetbox
from bokeh.plotting import figure, show, save
from bokeh.io import curdoc, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, Select
from bokeh.models import  Range1d, LinearColorMapper, ColorBar, LogColorMapper
from bokeh.models import GeoJSONDataSource
from bokeh.palettes import GnBu9, Magma6
import bokeh_winds as winds

In [30]:

plot = figure(plot_height=800, plot_width=1100, title='',
                           tools=["pan, reset, save, box_zoom, wheel_zoom, hover"],
                          x_axis_label='Longitude', y_axis_label='Latitude')

In [31]:
plot, color_bar = plot_data(u850_ens_mean[0], v850_ens_mean[0], precip_ens_mean[0], plot)
show(plot)